In [1]:
import urllib.request, json 
import pandas as pd

In [318]:
#Create Api call example
dataset_identifier="ICT_HH2"
filter_dict={"Country":["FRA"],
             "Indicator":[""], #Take all the indicators
             "Breakdown":["IND_TOTAL"]}
agency_name="all"
params={"detail":"dataonly"}

In [310]:
def makeAPICall(dataset_identifier,
                filter_dict,
                agency_name,
                params={},
                OECD_BASE_URL="http://stats.oecd.org/SDMX-JSON/data/"):
    # See https://data.oecd.org/api/sdmx-json-documentation/#d.en.330346
    """
    Create the url for the OECD API
    Arguments:
        dataset_identifier: string containing the name of the dataset
        filter_dict: dictionary containing the value of the variables to query (Country can be only one)
        agency_name: agency name to pass to the api  
    """

    filter_expression=""
    filter_list=[]

    if all(filter_dict.values())=="":
        filter_expression="all"
    else:
        for dim in filter_dict.keys():
            filter_list.append("+".join(filter_dict[dim]))
    filter_expression=".".join(filter_list)
    
    api_string=f"{OECD_BASE_URL}{dataset_identifier}/{filter_expression}/{agency_name}"
    if len(params)>0:
        api_string+="?"
        for par in params.keys():
            api_string+=(par+'='+params[par])

    return api_string

In [319]:
#json_url=makeAPICall(dataset_identifier,filter_dict,agency_name,params)
#print(json_url)

In [312]:
def getData(json_url):
    "Return a dict with the data"
    
    with urllib.request.urlopen(json_url) as url:
        data = json.loads(url.read().decode())
        #print(data)
    return data

In [315]:
def convertToPandas(data):
    col_names=[x["id"] for x in data["structure"]["dimensions"]["observation"][0]["values"] ]
    #print(col_names)
     
    dataset_dimensions = data["structure"]["dimensions"]["series"]
    print(f"Dataset dimensions:\t{len(dataset_dimensions)}")
    
    country = dataset_dimensions[0]["values"][0]["id"]
    print(f"Country:\t{country}")
    
    indicators=[x["id"] for x in dataset_dimensions[1]["values"]]
    print(f"Indicators:\t{indicators}")
    print(f"N indicators = {len(indicators)}")
    
    df_ind_list=[]
    for ind in range(0,len(indicators)):
        df = pd.DataFrame(data["dataSets"][0]["series"][f'0:{ind}:0']['observations'])
        df["Var"]=indicators[ind]
        df.set_index("Var",inplace=True)

        df_ind_list.append(df)
    
    df_res=pd.concat(df_ind_list,sort=False)
    df_res.columns = col_names
    
    df_res.reset_index(inplace=True)
    df_res["Country"]=country
    
    return df_res.set_index(["Country","Var"])

### Create final dataset

In [320]:
country_list = ["AUS","FRA","BEL"]

In [324]:
def createMultipleCountryDataset(country_list):
    """
    Create a dataset with multiple countries
    """

    df_to_concat = []

    for country in country_list:
        filter_dict["Country"] = [country]
        json_url=makeAPICall(dataset_identifier,filter_dict,agency_name,params)
        print(json_url)
        data=getData(json_url)
        df_to_concat.append(convertToPandas(data))
    
    res = pd.concat(df_to_concat,sort=False)
    res.columns = sorted(res.columns)
    
    return res

In [325]:
createMultipleCountryDataset(country_list)

http://stats.oecd.org/SDMX-JSON/data/ICT_HH2/AUS..IND_TOTAL/all?detail=dataonly
Dataset dimensions:	3
Country:	AUS
Indicators:	['C5A', 'C6A', 'C6B', 'C6C', 'C6D', 'D1A', 'D1B', 'D1E', 'D1K', 'D1Q', 'D1T', 'F1B', 'G1', 'G2H', 'G2K', 'G2O', 'IND', 'C5B', 'D1H', 'G2B', 'G2C', 'G2J', 'G2N']
N indicators = 23
http://stats.oecd.org/SDMX-JSON/data/ICT_HH2/FRA..IND_TOTAL/all?detail=dataonly
Dataset dimensions:	3
Country:	FRA
Indicators:	['C2A', 'C2B', 'C5A', 'C5B', 'C5B1', 'D1A', 'D1B', 'D1F', 'D1G', 'D1H', 'D1I', 'D1J', 'D1K', 'D1M', 'D1N', 'D1O', 'D1P', 'D1Q', 'D1R', 'D1T', 'D1U', 'F1', 'F1A', 'F1B', 'F1C', 'G1', 'G2A', 'G2B', 'G2C', 'G2G', 'G2H', 'G2I', 'G2J', 'G2K', 'G2M', 'G2N', 'G2O', 'H1A', 'H1B', 'H1C', 'H1D', 'H1E', 'H1F', 'H1G', 'H1H', 'H1I', 'H1J', 'H1K', 'H1L', 'H1M', 'I3', 'I3A', 'I3B', 'I3C', 'I3D', 'D1E', 'F4A', 'I8', 'C6E', 'I7A', 'I7B', 'F4J', 'F4G', 'G2DBIS', 'I6', 'D1W', 'I7C', 'F4F', 'F4B', 'I9']
N indicators = 70
http://stats.oecd.org/SDMX-JSON/data/ICT_HH2/BEL..IND_TOTAL/

2005  2006  2007  2008   2009    2010    2011     2012  2013  \
Country Var                                                                  
AUS     C5A  70.00  73.0  76.0  78.0  83.42  87.460     NaN      NaN   NaN   
        C6A  59.81  64.0  67.0  72.0    NaN     NaN     NaN      NaN   NaN   
        C6B  32.23  34.0  36.0  38.0    NaN     NaN     NaN      NaN   NaN   
        C6C  10.19  15.0  15.0  15.0    NaN     NaN     NaN      NaN   NaN   
        C6D  32.92  36.0  37.0  36.0    NaN     NaN     NaN      NaN   NaN   
...            ...   ...   ...   ...    ...     ...     ...      ...   ...   
BEL     I7A    NaN   NaN   NaN   NaN    NaN     NaN     NaN  29.9040   NaN   
        I7C    NaN   NaN   NaN   NaN    NaN     NaN     NaN  42.1599   NaN   
        I6     NaN   NaN   NaN   NaN    NaN     NaN     NaN  62.0330   NaN   
        F4B    NaN   NaN   NaN   NaN    NaN   1.752  1.5323   1.5461   NaN   
        F4I    NaN   NaN   NaN   NaN    NaN     NaN  2.5452   2.2102   NaN   

               2014    2015    2016    2017    2018  
Country Var                                          
AUS     C5A     NaN     NaN     NaN     NaN     NaN  
        C6A     NaN     NaN     NaN     NaN     NaN  
        C6B     NaN     NaN     NaN     NaN     NaN  
        C6C     NaN     NaN     NaN     NaN     NaN  
        C6D     NaN     NaN     NaN     NaN     NaN  
...             ...     ...     ...     ...     ...  
BEL     I7A     NaN     NaN     NaN     NaN     NaN  
        I7C     NaN     NaN     NaN     NaN     NaN  
        I6      NaN     NaN     NaN     NaN     NaN  
        F4B  1.1621  2.0813  1.6046  1.8011  1.1272  
        F4I     NaN  1.7545  2.4440  2.3409     NaN  

[163 rows x 14 columns]